# extract

> Extract Data from text using LLM

In [ ]:
#| default_exp extract

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import time
import uuid
from typing import List, Dict, Any, Optional, Tuple
from llm_data_extractor.models import Question, ExtractionResult, LLMConfig
from llm_data_extractor.prompt_builder import build_prompt
from llm_data_extractor.llm_client import get_llm_response, parse_llm_json_response, LLMClientError
from llm_data_extractor.validator import validate_answer
import logging

In [ ]:
#| export

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG) 

In [ ]:
#| export

def _process_question_response(question: Question, 
                              llm_responses: List[Dict], 
                              raw_response: str) -> ExtractionResult:
    """Process a single question's response from the LLM."""
    
    # Find matching response
    question_response = None
    for response in llm_responses:
        # Handle both dict and potentially malformed responses
        if isinstance(response, dict) and response.get('question_id') == question.id:
            question_response = response
            break
    
    if not question_response:
        return ExtractionResult(
            question_id=question.id,
            raw_answer="",
            parsed_answer=None,
            confidence=0.0,
            is_valid=False,
            validation_error="No response found for question"
        )
    
    # Safely extract answer and confidence
    try:
        raw_answer = str(question_response.get('answer', ''))
        confidence = float(question_response.get('confidence', 0.0))
    except (AttributeError, TypeError, ValueError) as e:
        logger.error(f"Error extracting answer/confidence: {str(e)}")
        return ExtractionResult(
            question_id=question.id,
            raw_answer="",
            parsed_answer=None,
            confidence=0.0,
            is_valid=False,
            validation_error=f"Response format error: {str(e)}"
        )
    
    # Validate the answer
    parsed_answer, is_valid, validation_error = validate_answer(raw_answer, question)
    
    return ExtractionResult(
        question_id=question.id,
        raw_answer=raw_answer,
        parsed_answer=parsed_answer,
        confidence=confidence,
        is_valid=is_valid,
        validation_error=validation_error if validation_error else None
    )



In [ ]:
#| export

def _fallback_parse_response(raw_response: str, questions: List[Question]) -> List[Dict]:
    """
    Fallback parser for when structured JSON parsing fails.
    Attempts to extract answers using simple text parsing.
    """
    
    responses = []
    lines = raw_response.split('\n')
    
    for question in questions:
        # Look for patterns like "Question 1:", "Q1:", question ID, etc.
        answer = ""
        confidence = 0.0
        
        for i, line in enumerate(lines):
            line = line.strip()
            
            # Check if this line references our question
            if (question.id in line or 
                f"Question {questions.index(question) + 1}" in line or
                question.text[:20] in line):
                
                # Look ahead for the answer
                for j in range(i + 1, min(i + 5, len(lines))):
                    next_line = lines[j].strip()
                    if next_line and not next_line.startswith('Q'):
                        answer = next_line
                        break
                
                break
        
        responses.append({
            'question_id': question.id,
            'answer': answer,
            'confidence': confidence
        })
    
    return responses

In [ ]:
#| export

def extract_data(source_text: str, 
                questions: List[Question], 
                llm_config: LLMConfig,
                source_id: Optional[str] = None,
                batch_id: Optional[str] = None) -> Tuple[List[ExtractionResult], Dict[str, Any]]:
    """
    Extract structured data from unstructured text using LLM.
    
    Args:
        source_text: The text to extract data from
        questions: List of questions to answer
        llm_config: LLM configuration
        source_id: Optional identifier for the source
        batch_id: Optional batch identifier
        
    Returns:
        Tuple of (extraction_results, metadata)
    """
    
    if not questions:
        return [], {'error': 'No questions provided'}
    
    # Generate batch_id if not provided
    if not batch_id:
        batch_id = str(uuid.uuid4())
    
    start_time = time.time()

    print('-'*80); print('QUESTIONS'); print('-'*80); print(questions); print('-'*80);
    
    try:
        # Step 1: Build prompt
        prompt = build_prompt(questions, source_text)
        
        # Step 2: Get LLM response
        raw_response = get_llm_response(prompt, llm_config)
        print('-'*80); print('RAW RESPONSE'); print('-'*80)
        print(raw_response)
        print('-'*80)
        # Step 3: Parse JSON response
        try:
            parsed_response = parse_llm_json_response(raw_response)
            llm_responses = parsed_response.get('responses', [])
        except Exception as e:
            logger.warning(f"Failed to parse structured JSON response: {str(e)}")
            # Fallback: try to extract answers from unstructured response
            llm_responses = _fallback_parse_response(raw_response, questions)
        
        # Step 4: Validate and create results
        results = []
        for question in questions:
            result = _process_question_response(question, llm_responses, raw_response)
            results.append(result)
        
        # Step 5: Create metadata
        processing_time = time.time() - start_time
        metadata = {
            'batch_id': batch_id,
            'source_id': source_id,
            'processing_time_seconds': processing_time,
            'total_questions': len(questions),
            'raw_llm_response': raw_response,
            'prompt_used': prompt,
            'llm_config': llm_config.__dict__,
        }
        
        return results, metadata
        
    except Exception as e:
        # Handle catastrophic failures
        processing_time = time.time() - start_time
        error_results = []
        
        for question in questions:
            error_result = ExtractionResult(
                question_id=question.id,
                raw_answer="",
                parsed_answer=None,
                confidence=0.0,
                is_valid=False,
                input_row_id=None,
                validation_error=f"Processing failed: {str(e)}"
            )
            error_results.append(error_result)
        
        metadata = {
            'batch_id': batch_id,
            'source_id': source_id,
            'processing_time_seconds': processing_time,
            'total_questions': len(questions),
            'error': str(e),
            'llm_config': llm_config.__dict__
        }
        
        return error_results, metadata


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()